
# 🔧 Setup for `spotlightcurve`

This notebook is prepped to work in two ways:

**Option A (preferred): pip-install from GitHub**  
```python
%pip install --no-build-isolation --no-deps "git+https://github.com/ArifSolmaz/arifsolmaz.github.io@main"
# then (runtime deps, optional if already present in Colab)
%pip install -q -r https://raw.githubusercontent.com/ArifSolmaz/arifsolmaz.github.io/main/requirements.txt
```

**Option B (fallback if A fails or you're behind a proxy): clone & add to path**  
```python
!rm -rf arifsolmaz.github.io
!git clone https://github.com/ArifSolmaz/arifsolmaz.github.io.git
%cd arifsolmaz.github.io
import sys, os
sys.path.append(os.getcwd())
```

**Local Mac (outside Colab) virtualenv quickstart**  
```bash
cd /path/to/arifsolmaz.github.io
python3 -m venv .venv
source .venv/bin/activate
pip install --upgrade pip setuptools wheel
pip install -e .
python -c "import spotlightcurve, sys; print('OK:', spotlightcurve.__version__)"
```


In [ ]:

# Try to install from GitHub (no build isolation to avoid downloading build deps)
try:
    # Install the package itself (no build isolation / deps)
    %pip install --no-build-isolation --no-deps "git+https://github.com/ArifSolmaz/arifsolmaz.github.io@main" -q
    # Install runtime dependencies (best effort)
    %pip install -q -r https://raw.githubusercontent.com/ArifSolmaz/arifsolmaz.github.io/main/requirements.txt
    INSTALLED_MODE = "pip_git"
except Exception as e:
    print("pip install from GitHub failed, falling back to clone:", e)
    INSTALLED_MODE = None

if INSTALLED_MODE is None:
    # Fallback: clone and add repo to path
    import os, sys, subprocess, textwrap
    try:
        subprocess.run(["bash","-lc","rm -rf arifsolmaz.github.io && git clone https://github.com/ArifSolmaz/arifsolmaz.github.io.git"], check=True)
        %cd arifsolmaz.github.io
        import os, sys
        sys.path.append(os.getcwd())
        INSTALLED_MODE = "clone_path"
    except Exception as e:
        print("Fallback clone also failed:", e)
        INSTALLED_MODE = "failed"

print("Install mode:", INSTALLED_MODE)


In [ ]:

# Import test
try:
    import spotlightcurve as sc
    print("spotlightcurve version:", getattr(sc, "__version__", "unknown"))
    print("module file:", getattr(sc, "__file__", "unknown"))
except Exception as e:
    print("Import failed:", e)
    raise


In [ ]:

# Quick demo: simulate and plot a spot-modulated light curve
import numpy as np
import matplotlib.pyplot as plt

try:
    from spotlightcurve import core
    t = np.linspace(0, 10, 2000)
    # If simulate_spot_curve exists in your package, use it; else make a simple cosine
    if hasattr(core, "simulate_spot_curve"):
        flux = core.simulate_spot_curve(t, period=3.2, amplitude=0.01, noise=5e-4, seed=1)
    else:
        phase = 2*np.pi*(t/3.2)
        flux = 1.0 - 0.01*np.cos(phase)
    plt.figure()
    plt.plot(t, flux)
    plt.xlabel("Time [days]")
    plt.ylabel("Relative flux")
    plt.title("SpotlightCurve quick demo")
    plt.show()
except Exception as e:
    print("Demo failed:", e)


# SpotLightCurve Colab Tutorial

This notebook demonstrates how to install the **SpotLightCurve** package in Google Colab and run an end-to-end analysis of the HAT-P-36 system using TESS Sector 49 photometry.

## 1. Install SpotLightCurve and dependencies

Install the latest version of SpotLightCurve directly from GitHub. The `--quiet` flag keeps the output manageable in Colab logs.

In [ ]:
!pip install --quiet "git+https://github.com/arifsolmaz/arifsolmaz.github.io@main"

## 2. Import libraries

After the install step completes, import NumPy, PyMC, Lightkurve, and the SpotLightCurve helpers used throughout the workflow. If Colab prompts you to restart the runtime, do so before running this cell.

In [ ]:
import numpy as np
import arviz as az
import pymc as pm
import lightkurve as lk
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive

from spotlightcurve.io import load_tess
from spotlightcurve.preprocess import quality_mask, apply_mask
from spotlightcurve.model import build_gp_transit_model

## 3. Download and clean the TESS light curve

Fetch the Sector 49 light curve for HAT-P-36 using Lightkurve, then apply a simple sigma-clipping quality mask.

In [ ]:
target = "HAT-P-36"
sector = 49

lc = load_tess(target, sector=sector)
mask = quality_mask(lc.flux)
lc_clean = apply_mask(lc, mask)

lc_clean.metadata

## 4. Retrieve published planetary ephemerides

Use the NASA Exoplanet Archive to obtain the orbital period and transit mid-point for HAT-P-36 b. TESS works in the BTJD time system, so convert from BJD.

In [ ]:
ephem = NasaExoplanetArchive.query_planet("HAT-P-36 b")
period_days = ephem["pl_orbper"][0]
t0_bjd = ephem["pl_tranmid"][0]
t0_btjd = t0_bjd - 2457000.0

print(f"Period = {period_days:.6f} d")
print(f"Transit mid-point (BTJD) = {t0_btjd:.6f}")

## 5. Estimate a stellar rotation-period prior

Compute a Lomb-Scargle periodogram with Lightkurve to get an initial guess for the stellar rotation period, which informs the Gaussian-process kernel.

In [ ]:
lk_lc = lk.LightCurve(time=lc_clean.time, flux=lc_clean.flux)
pg = lk_lc.to_periodogram(method="lombscargle", minimum_period=1, maximum_period=25)
rot_period_guess = pg.period_at_max_power.value

print(f"Rotation-period prior ≈ {rot_period_guess:.2f} d")

## 6. Build and sample the GP + transit model

Set up the SpotLightCurve probabilistic model that combines a rotation-tailored Gaussian process with the planetary transit signal. Sampling with PyMC's NUTS sampler can take a few minutes on Colab. Feel free to adjust the number of draws for quicker tests.

In [ ]:
model = build_gp_transit_model(
    time=lc_clean.time,
    flux=lc_clean.flux,
    flux_err=lc_clean.flux_err,
    period=period_days,
    t0=t0_btjd,
    rot_period_prior=rot_period_guess,
)

with model:
    trace = pm.sample(
        draws=1000,
        tune=1000,
        chains=2,
        target_accept=0.9,
        return_inferencedata=True,
    )

## 7. Summarize planetary and spot parameters

ArviZ summarises posterior estimates for key planetary and stellar-activity parameters.

In [ ]:
planet_vars = ["period", "t0", "r", "impact"]
spot_vars = ["rot_period", "log_amp", "mixing"]

planet_summary = az.summary(trace, var_names=planet_vars)
spot_summary = az.summary(trace, var_names=spot_vars)

planet_summary, spot_summary

## 8. Visual diagnostics

Inspect trace plots for convergence and revisit the periodogram to see the rotation signal.

In [ ]:
az.plot_trace(trace, var_names=planet_vars + spot_vars);
pg.plot();

## 9. (Optional) Save outputs to Google Drive

Mount Google Drive and store the posterior samples for future analysis.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# az.to_netcdf(trace, '/content/drive/MyDrive/spotlightcurve/hatp36_sector49_trace.nc')